In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# #-------------------------------------------------#
#     Topic American Express Default Prediction    
# #-------------------------------------------------#

Goal: We aims to build a robust model to predict default credit payement of client

Data Insights: 
The data we use has 190 columns. it contains both categorical and Numerical values. 

The following column name has categorical value and the remainder is Numerical 

['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'].

The Numerical values are normalized 



## Dependancies 

In [28]:
import pandas as pd
import missingno as mno

## Data Loading

In [3]:
X_train= pd.read_feather("../input/feather-datasets-amex-default-prediction/train_data.ftr",columns=None, use_threads=True, storage_options=None)



In [ ]:
Y_train=pd.read_csv("../input/amex-default-prediction/train_labels.csv")

In [5]:
Test=pd.read_feather("../input/feather-datasets-amex-default-prediction/test_data.ftr",columns=None, use_threads=True, storage_options=None)

### Data insights

In [4]:
X_train.head(5)

## List and chunk all columns according to their type

In [4]:
from dataclasses import dataclass
from typing import List 


@dataclass
class OutVerificator():
    columnsWhereTypeObject : List[str]
    columnsWhereTypeCategory : List[str]
    columnsWhereFloat16 : List[str]
    columnsWhereOther : List[str]
      
    

class typeVerificator():
    def __init__(self):
        pass
    
    def Verificator(self,df):
        columnsWhereTypeObject=[list(df.columns)[columns_Index] for columns_Index in range(len(df.columns)) if str(list(df.dtypes)[columns_Index])=="object" ]
        columnsWhereTypeCategory=[list(df.columns)[columns_Index] for columns_Index in range(len(df.columns)) if str(list(df.dtypes)[columns_Index])=="category" ]
        columnsWhereFloat16=[list(df.columns)[columns_Index] for columns_Index in range(len(df.columns)) if str(list(df.dtypes)[columns_Index])=='float16' ]
        columnsWhereOther=[ list(df.columns)[columns_Index] for columns_Index in range(len(df.columns)) if (str(list(df.dtypes)[columns_Index])!='float16' 
                                                            and str(list(df.dtypes)[columns_Index])!="category" and str(list(df.dtypes)[columns_Index])!="object")]
        
        return OutVerificator(columnsWhereTypeObject=columnsWhereTypeObject,
                              columnsWhereTypeCategory=columnsWhereTypeCategory,
                              columnsWhereFloat16=columnsWhereFloat16,
                              columnsWhereOther=columnsWhereOther)
        
        
        


In [5]:
parser=typeVerificator()
Response=parser.Verificator(X_train)

In [6]:
#### If it not assertion Error it means that assertion is verified
assert (len(Response.columnsWhereTypeObject)+len(Response.columnsWhereTypeCategory) + len (Response.columnsWhereFloat16) + len(Response.columnsWhereOther))==len(X_train.columns)

In [10]:
Response.columnsWhereTypeCategory

## Categorical columns handling
We want to process the categorical columns by using a dummy variable and combining the result with the data recovery after removing the object columns. 

['D_63','D_64','D_66', 'D_68','B_30','B_38','D_114','D_116','D_117','D_120','D_126']

In [8]:
from sklearn.preprocessing import MinMaxScaler

class Categorical2Numerical():
    """ 
    Convert categorical variable 2 Numerical using dummy variable
    """
    def __init__(self,df):
        parser=typeVerificator()
        self.Response=parser.Verificator(df)
        self.df=df
        
    def __str__(self):
        return "We aims to convert categorical variable 2 Numerical using dummy variable"
    
    def categorical2numerical(self):
        df=pd.get_dummies(self.df[self.Response.columnsWhereTypeCategory],dummy_na=True)
        
        scaler=MinMaxScaler()
        scaled= scaler.fit_transform(df)
        
        return pd.DataFrame(scaled, columns=df.columns)
        
        
        
        
     

In [9]:
parser=Categorical2Numerical(X_train)
parser.categorical2numerical()

## Merged the float, int and the last columns founded using dummy variable 

In [10]:
class Merging(Categorical2Numerical):
    def __init__(self,df):
        super().__init__(df)
   
    def __str__(self):
        return "Merge DataFrames"
    
    def merging(self):
 
        return pd.concat([self.df[ self.Response.columnsWhereFloat16],self.df[ self.Response.columnsWhereOther],self.categorical2numerical()], axis = 1) 
        

In [11]:
parser=Merging(X_train)
X_trainMerged=parser.merging()

In [12]:
X_trainMerged

## NaN values Processing

In [13]:
@dataclass
class outNanSupervisor():
    columnsNan : List[str]
    columnsNotNan : List[str]
        
    

class NanSupervisor():
    """
    This function takes a dataframe and returns an object of type list.
    This object contains The names of the columns in which there is
    Nan and The names of the columns in which there is not Nan.
    """
    def __init__(self,df):
        self.df=df
    
    def nansupervisor(self):
        columnsNan=[column for column in list(self.df.columns) if self.df[column].isnull().sum()>0 ]
        columnsNotNan=list(set(list(self.df.columns)).difference(set(columnsNan)))
        
        return outNanSupervisor(columnsNan=columnsNan,columnsNotNan=columnsNotNan)
        
        
    

In [44]:
parser=NanSupervisor(X_trainMerged)
Response=parser.nansupervisor()
Response.columnsNan

In [33]:
mno.bar(X_trainMerged[['P_2','B_2','S_3','D_41','B_3','D_42','D_43','D_44','D_45','D_46']])

In [27]:
Response.columnsNotNan

In [35]:
mno.bar(X_trainMerged[['D_63_CL','D_75','D_68_4.0','B_38_nan','B_9','B_30_1.0','R_3','D_68_2.0','D_58','R_6','B_14','S_20','D_64_O','D_68_5.0','R_22']],color="blue")

How i want to process the Nan values 
* D_* = Delinquency variables
* S_* = Spend variables
* P_* = Payment variables
* B_* = Balance variables
* R_* = Risk variables

To process the Nan inside the dataframe, I want to do it as follows:

* D_* = Delinquency variables (Most frequence values)
* S_* = Spend variables (Mean values)
* P_* = Payment variables (Mean values)
* B_* = Balance variables (Median values)
* R_* = Risk variables (Mean values)

In [84]:
import re 
from  sklearn.impute import SimpleImputer
    
class DataCleaner(NanSupervisor):
    """
    This function Handling Nan values 
    and return the dataframe without Nan
    """
    
    def __init__(self,df):
        super().__init__(df)
    
    def datacleaner(self):
        "chunking"
        Response=self.nansupervisor()
        AllcolumnsNan=Response.columnsNan
        DelinquencyNan=[columnDelinquency for columnDelinquency in AllcolumnsNan if re.search("D_\d+", columnDelinquency) ]
        SpendNan=[columnSpend for columnSpend in AllcolumnsNan if re.search("S_\d+", columnSpend) ]
        PaymentNan=[columnPayment for columnPayment in AllcolumnsNan if re.search("P_\d+", columnPayment) ]
        BalanceNan=[columnBalance for columnBalance in AllcolumnsNan if re.search("B_\d+", columnBalance) ]
        RiskNan=[columnRisk for columnRisk in AllcolumnsNan if re.search("R_\d+", columnRisk) ]
        
        assert (len(DelinquencyNan) + len(SpendNan) + len(PaymentNan) + len(BalanceNan) +len( RiskNan))==len(AllcolumnsNan)
        
        #if assertion error, review this function
        # List columns Mean 
        SPR=list((set(SpendNan).union(PaymentNan)).union(set(RiskNan)))
        
        for columns in SPR:
            self.df[columns].fillna(self.df[columns].mean(),inplace=True)
#             parser=SimpleImputer(missing_values=0,strategy="mean")
#             self.df[columns]=parser.fit_transform(self.df[columns].values.reshape(len(self.df[columns]),1))
            
        # List columns Median
        for columns in BalanceNan:
            self.df[columns].fillna(self.df[columns].median(),inplace=True)
#             parser=SimpleImputer(missing_values=0,strategy="median")
#             self.df[columns]=parser.fit_transform(self.df[columns].values.reshape(len(self.df[columns]),1))
            
        # List columns most_frequent 
        for columns in DelinquencyNan:
            self.df[columns].fillna(self.df[columns].median(),inplace=True)
#             parser=SimpleImputer(missing_values=0,strategy="most_frequent ")
#             self.df[columns]=parser.fit_transform(self.df[columns].values.reshape(len(self.df[columns]),1))
            
        return self.df
        
        

In [85]:
parser=DataCleaner(X_trainMerged)
X_trainwithoutna=parser.datacleaner()


In [86]:
X_trainwithoutna